<a href="https://colab.research.google.com/github/Jcain17/Intro_to_ML/blob/main/ML_HW4_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import seaborn as sns; sns.set()

drive.mount('/content/drive')

file_path = '/content/drive/My Drive/cancer.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))
dataset.head()

Mounted at /content/drive


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [2]:
def binmap(x):
   return x.map({'M': 1, 'B': 0})

dataset[['diagnosis']] = dataset[['diagnosis']].apply(binmap)
dataset.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
x_vars = np.arange(2,31,1)
X = dataset.iloc[:, x_vars].values
Y = dataset.iloc[:, 1].values

X[0:2]

array([[1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
        3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
        8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
        3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
        1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, 1.326e+03, 8.474e-02, 7.864e-02,
        8.690e-02, 7.017e-02, 1.812e-01, 5.667e-02, 5.435e-01, 7.339e-01,
        3.398e+00, 7.408e+01, 5.225e-03, 1.308e-02, 1.860e-02, 1.340e-02,
        1.389e-02, 3.532e-03, 2.499e+01, 2.341e+01, 1.588e+02, 1.956e+03,
        1.238e-01, 1.866e-01, 2.416e-01, 1.860e-01, 2.750e-01]])

In [4]:
# Establish PCA and SVC before slitting data
svc = SVC(kernel='linear', class_weight='balanced')

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 60)

sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [6]:
k = 5
pca = PCA(n_components=k)

pc_train = pca.fit_transform(X_train)
pc_test = pca.fit_transform(X_test)

svc = SVC(kernel='linear', C=1E5)
svc.fit(pc_train, Y_train)

Y_pred = svc.predict(pc_test)

print("Iteration: ", k)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))
print("Precision:",metrics.precision_score(Y_test, Y_pred))
print("Recall:",metrics.recall_score(Y_test, Y_pred))
print("F1:",metrics.f1_score(Y_test, Y_pred))
print('')

Iteration:  5
Accuracy: 0.9649122807017544
Precision: 0.9736842105263158
Recall: 0.925
F1: 0.9487179487179489



In [7]:
pca = PCA(n_components=k)

pc_train = pca.fit_transform(X_train)
pc_test = pca.fit_transform(X_test)

svc = SVC(kernel='rbf', C=1E10, gamma = 'scale')
svc.fit(pc_train, Y_train)

Y_pred = svc.predict(pc_test)

print("Iteration: ", k)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))
print("Precision:",metrics.precision_score(Y_test, Y_pred))
print("Recall:",metrics.recall_score(Y_test, Y_pred))
print("F1:",metrics.f1_score(Y_test, Y_pred))
print('')

Iteration:  5
Accuracy: 0.868421052631579
Precision: 0.8378378378378378
Recall: 0.775
F1: 0.8051948051948051



In [8]:
pca = PCA(n_components=k)

pc_train = pca.fit_transform(X_train)
pc_test = pca.fit_transform(X_test)

svc = SVC(kernel='poly', C=1E10, degree = 3)
svc.fit(pc_train, Y_train)

Y_pred = svc.predict(pc_test)

print("Iteration: ", k)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))
print("Precision:",metrics.precision_score(Y_test, Y_pred))
print("Recall:",metrics.recall_score(Y_test, Y_pred))
print("F1:",metrics.f1_score(Y_test, Y_pred))
print('')

Iteration:  5
Accuracy: 0.9210526315789473
Precision: 0.918918918918919
Recall: 0.85
F1: 0.8831168831168831



In [9]:
pca = PCA(n_components=k)

pc_train = pca.fit_transform(X_train)
pc_test = pca.fit_transform(X_test)

svc = SVC(kernel='sigmoid', C=1E10)
svc.fit(pc_train, Y_train)

Y_pred = svc.predict(pc_test)

print("Iteration: ", k)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))
print("Precision:",metrics.precision_score(Y_test, Y_pred))
print("Recall:",metrics.recall_score(Y_test, Y_pred))
print("F1:",metrics.f1_score(Y_test, Y_pred))
print('')

Iteration:  5
Accuracy: 0.9035087719298246
Precision: 0.8717948717948718
Recall: 0.85
F1: 0.8607594936708861

